# Résumé automatique
Il existe deux façon de créer un résmué de texte : 
  - le résumé par extraction, qui recherche et extrait les phrases les plus importantes
  - le résumé par abstraction qui ré-écrit les phrases les plus importabtes du texte.

Ici, on s'intéresse au résumé par extraction.

<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/resume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Pour ouvrir sous Colab"/></a>

---
Prenons un texte, un extrait des Misérables (chapitre 1 - Fontine) de V. Hugo :

In [44]:
text="""En 1815, M. Charles-François-Bienvenu Myriel était évêque de Digne. 
C'était un vieillard d'environ soixante-quinze ans; il occupait le siège de Digne depuis 1806. 
Quoique ce détail ne touche en aucune manière au fond même de ce que nous avons à raconter, il n'est peut-être pas inutile, ne fût-ce que pour être exact en tout, d'indiquer ici les bruits et les propos qui avaient couru sur son compte au moment où il était arrivé dans le diocèse. 
Vrai ou faux, ce qu'on dit des hommes tient souvent autant de place dans leur vie et surtout dans leur destinée que ce qu'ils font. 
M. Myriel était fils d'un conseiller au parlement d'Aix; noblesse de robe. 
On contait de lui que son père, le réservant pour hériter de sa charge, l'avait marié de fort bonne heure, à dix-huit ou vingt ans, suivant un usage assez répandu dans les familles parlementaires . 
Charles Myriel, nonobstant ce mariage, avait, disaiton, beaucoup fait parler de lui. 
Il était bien fait de sa personne, quoique d'assez petite taille, élégant, gracieux, spirituel; toute la première partie de sa vie avait été donnée au monde et aux galanteries.  
La révolution survint, les événements se précipitèrent, les familles parlementaires décimées, chassées, traquées, se dispersèrent.  
M. Charles Myriel, dès les premiers jours de la révolution, émigra en Italie.  
Sa femme y mourut d'une maladie de poitrine dont elle était atteinte depuis longtemps. 
Ils n'avaient point d'enfants.  Que se passa-t-il ensuite dans la destinée de M. Myriel?  
L'écroulement de l'ancienne société française, la chute de sa propre famille, les tragiques spectacles de 93, plus effrayants encore peut-être pour les émigrés qui les voyaient de loin avec le grossissement de l'épouvante, firent-ils germer en lui des idées de renoncement et de solitude?  
Fut-il, au milieu d'une de ces distractions et de ces affections qui occupaient sa vie, subitement atteint d'un de ces coups mystérieux et terribles qui viennent quelquefois renverser, en le frappant au cœur, l'homme que les catastrophes publiques n'ébranleraient pas en le frappant dans son existence et dans sa fortune? 
Nul n'aurait pu le dire; tout ce qu'on savait, c'est que, lorsqu'il revint d'Italie, il était prêtre.  
En 1804 M. Myriel était curé de Brignolles. Il était déjà vieux, et vivait dans une retraite profonde. """

---
### on souhaite ***résumer ce texte !*** 

On utilise un bibliothèque dédié au traitement de texte.

---
## Spacy : un outil de traitement de langage naturel
Spacy est une bibliothèque assez complète de NLP (Natural Language Processing).

Elle est déjà installée dans l'environnement Colab. On peut ainsi l'importer :  

In [4]:
import spacy

## 1. Le corpus
Il existe des corpus pour de nombreuses langues, la langue anglaise étant traitée de façon plus complète.

Pour chaque langue, il existe en général 3 versions de corpus long, médium et petit.
 - Par exemple, en anglais, on chargera les corpus en_core_web_lg, en_core_web_md ou en_core_web_sm.


Ici, on s'intéresse à la langue français et si le corpus long français n'a pas été installé, on le télécharge avec la commande suivante :  

In [ ]:
!python -m spacy download fr_core_news_lg 

In [5]:
import fr_core_news_lg

On peut alors charger le corpus en mémoire : 

In [9]:
nlp = fr_core_news_lg.load()

---
## 2. Les mots "inutiles"
Pour traiter le texte, on peut choisir de ne pas considérer les mots des liaisons (qui n'apportent pas au sens d'une phrase), ce sont les stop_words.

Spacy, comme d'autres nlp, dispose de listes de stopwords que l'on peut augmenter manuellement.

In [10]:
#récupérer une liste de mots "inutiles" dans le corpus français : 
from spacy.lang.fr.stop_words import STOP_WORDS
stopwords=list(STOP_WORDS)

---
## 3. Ponctuation
Pour traiter le texte, on peut choisir de ne pas considérer  les signes de ponctuation.

Python fournit une liste de ces signes que l'on peut augmenter avec d'autres signes : 

In [11]:
from string import punctuation
punctuation = punctuation+ '\n'

## 4. Chargement du texte
La commande `nlp`de Spacy charge un texte et l'analyse. 
Un *document* résulte de cette analyse : 

In [41]:
def tokenize(text:str)->spacy.tokens.doc.Doc:
    doc = nlp(text)
    return doc

In [42]:
doc = tokenize(text)

## 5. Construction du résumé par extraction

  - Pour construire un résumé par extraction, on ne garde que les phrases les plus importantes.

  - Une phrase est importante si elle contient des mots importants.

  - Un mot est d'autant plus important s'il apparait plusieurs fois dans le texte.

### 5.1. Créaction d'un sac de mots
On souhaite créer une map, un dictionnaire python contenant en clé un mot, et en valeur son nombre d'apparartions dans le texte.



### Notes sur les dictionnaires pythons 
Ce sont des tables (clé:valeur) : 

In [16]:
## exemple de dictionnaire : 
eurovision = {}
eurovision['italie'] = 250
eurovision['estonie'] = 220
eurovision['france'] = 2


In [17]:
eurovision

{'italie': 250, 'estonie': 220, 'france': 2}

In [18]:
eurovision['france'] += 1
eurovision

{'italie': 250, 'estonie': 220, 'france': 3}

In [20]:
eurovision.keys()

dict_keys(['italie', 'estonie', 'france'])

---
### Note sur les documents de spacy

Un `doc` Spacy est un itérable qui peut se parcourir comme une liste : 


In [27]:
for w in doc: print(w.text, end=' ')

En 1815 , M. Charles - François - Bienvenu Myriel était évêque de Digne . 
 C' était un vieillard d' environ soixante-quinze ans ; il occupait le siège de Digne depuis dix-huit cent six . 
 Quoique ce détail ne touche en aucune manière au fond même de ce que nous avons à raconter , il n' est peut-être pas inutile , ne fût -ce que pour être exact en tout , d' indiquer ici les bruits et les propos qui avaient couru sur son compte au moment où il était arrivé dans le diocèse . 
 Vrai ou faux , ce qu' on dit des hommes tient souvent autant de place dans leur vie et surtout dans leur destinée que ce qu' ils font . 
 M. Myriel était fils d' un conseiller au parlement d' Aix ; noblesse de robe . 
 On contait de lui que son père , le réservant pour hériter de sa charge , l' avait marié de fort bonne heure , à dix-huit ou vingt ans , suivant un usage assez répandu dans les familles parlementaires . 
 Charles Myriel , nonobstant ce mariage , avait , disaiton , beaucoup fait parler de lui . 
 Il 

Les mots (`Token') de spacy disposent de champs : 
  - text (le texte du token)
  - lemma_ (le terme initial (ex. infinitif du verbe, ....)
  - et d'autres permettant de qualifier le mot (sujet, personne, style de conjugaison, suffixe, prefixe, .....)

Les textes de python peuvent être convertis en minuscule par la fonction `lower()` (`"BONJOUR".lower()` donne `"bonjour"`).

**QUESTION 1. Créer la fonction `calcul_de_frequences(doc:spacy.tokens.doc.Doc)->dict`** qui prend un document Spacy en entrée et donne en retour un dictionnaire contenant les mots utiles et leurs fréquences d'apparition (nombres d'apparition/le plus haut score d'apparition d'un mot) dans le texte.

In [47]:
### compute  the frequence of use of significant words
### @return a dictionary (word, nb of use/max nb of use )
def calcul_de_frequences(doc:spacy.tokens.doc.Doc)->dict:
  #TODO: completer
    return mots_frequences

---
Spacy dispose d'une fonction permettant de récupérer les phrases d'un texte.
Cette fonction n'est pas parfaite, le fait de couper à chaque signe '.' n'est pas réaliste. 
Par exemple "M. Dupont enquête." n'est d'une phrase et non 2...

Spacy prend en compte ce cas et d'autres.. Cependant des tournures de phrase peuvent parfois être mal détecter..  
*Vous vérifierez, seulement par la suite pour affiner le TP, si Spacy a bien détecter les phrases du texte. 
Si ce n'est pas le cas, corriger en modifiant légèrement le texte...*

Voici la commande pour récupérer les phrases : `doc.sents`

In [45]:
i=0
for phrase in doc.sents:
  i+=1
  print(f'phrase {i} : {phrase}')

phrase 1 : En 1815, M. Charles-François-Bienvenu Myriel était évêque de Digne. 

phrase 2 : C'était un vieillard d'environ soixante-quinze ans;
phrase 3 : il occupait le siège de Digne depuis 1806. 

Quoique ce détail ne touche en aucune manière au fond même de ce que nous avons à raconter, il n'est peut-être pas inutile, ne fût-ce que pour être exact en tout, d'indiquer ici les bruits et les propos qui avaient couru sur son compte au moment où il était arrivé dans le diocèse. 

phrase 4 : Vrai ou faux,
phrase 5 : ce qu'on dit des hommes tient souvent autant de place dans leur vie et surtout dans leur destinée que ce qu'ils font. 

phrase 6 : M. Myriel était fils d'un conseiller au parlement d'Aix;
phrase 7 : noblesse de robe. 

phrase 8 : On contait de lui que son père, le réservant pour hériter de sa charge, l'avait marié de fort bonne heure, à dix-huit ou vingt ans, suivant un usage assez répandu dans les familles parlementaires . 
Charles Myriel, nonobstant ce mariage, avait, disai

---
**QUESTION 2. Créer la fonction `calculer_scores_phrases(doc:spacy.tokens.doc.Doc, frequences_de_mots:dict)->dict`** qui prend en entrée : 
  - un document Spacy 
  - un dictionnaires des mots utiles et de leurs nombre d'apparition
et donne en sortie un dictionnaire avec comme clé : le no de la phrase dans le doc, et en valeur : la somme des valeurs des mots importants qui la compose.[texte du lien](https://)

In [60]:
### compute  the score of each sentence
### @return a dictionary (sentence, sum of word values)
def compute_sentence_scores(doc:spacy.tokens.doc.Doc, word_frequencies:dict)->dict:
  #TODO: Compléter
    return phrases_scores

---
*Il ne reste plus qu'à* créer le résumé.. 

**QUESTION 3. Créer la fonction `calculer_résumé(texte:str, ratio:float)->str`** qui prend en entrée : 
  - un long `texte`
  - un `ratio` entre 0 et 1
et retourne le texte initiale résumé à `ratio`%.

Pour créer le résumé, on trie les phrases par ordre de valeur, on en récupère `ratio`% du total des phrases, et on les affiches dans l'ordre de leur no.

In [81]:
from heapq import nlargest

def calculer_resume(texte:str, ratio:float)->str:
  #TODO....
    return resume

In [82]:
summary = calculer_resume(text, 0.4)
print(f'summary={summary}')

len(sentence_tokens)=19
select_length=7
summary=[0, 1, 4, 9, 10, 13, 16]
summary=En 1815, M. Charles-François-Bienvenu Myriel était évêque de Digne. 
C'était un vieillard d'environ soixante-quinze ans;ce qu'on dit des hommes tient souvent autant de place dans leur vie et surtout dans leur destinée que ce qu'ils font. 
toute la première partie de sa vie avait été donnée au monde et aux galanteries.  
La révolution survint, les événements se précipitèrent, les familles parlementaires décimées, chassées, traquées, se dispersèrent.  
M. Charles Myriel, dès les premiers jours de la révolution, émigra en Italie.  
Que se passa-t-il ensuite dans la destinée de M. Myriel?  
tout ce qu'on savait, c'est que, lorsqu'il revint d'Italie, il était prêtre.  

